In [5]:
import modules
import importlib
importlib.reload(modules)

# design variables
x = {'capture_width':10,
    'capture_width_ratio':0.5,
    'wec_type':"point absorber",
    'pen_diameter':1,
    'pen_height':0.5 }

# parameters
p = {'wave_height':2,
    'wave_period':5 }

power, cost_per_yield, carrying_capacity = modules.sim(x,p)

